In [46]:
import pandas as pd
import numpy as np
from scipy import stats
import math
from pandas import Series
import random
from mlxtend.evaluate import permutation_test

In [47]:
male_data = pd.read_csv('Male Data.csv')
female_data = pd.read_csv('Female Data.csv')

In [48]:
male_data.set_index ('REF_DATE', inplace = True)
female_data.set_index ('REF_DATE', inplace = True)

In [49]:
male_data.drop(labels=['DGUID','UOM','UOM_ID','SCALAR_FACTOR','SCALAR_ID','VECTOR','COORDINATE','STATUS','SYMBOL','TERMINATED','DECIMALS','Age group'], axis=1, inplace=True)
female_data.drop(labels=['DGUID','UOM','UOM_ID','SCALAR_FACTOR','SCALAR_ID','VECTOR','COORDINATE','STATUS','SYMBOL','TERMINATED','DECIMALS','Age group'], axis=1, inplace=True)

In [50]:
male_data.rename(columns = {'GEO':'Province', 'Duration of unemployment':'Duration', 'VALUE':'Total_Unemployed'}, inplace = True)
female_data.rename(columns = {'GEO':'Province', 'Duration of unemployment':'Duration', 'VALUE':'Total_Unemployed'}, inplace = True)

In [51]:
male_data = male_data[male_data.Duration != '26 weeks']
female_data = female_data[female_data.Duration != '26 weeks']

In [52]:
male_data["Total_Unemployed"].fillna(male_data.groupby("Province")["Total_Unemployed"].transform("mean"), inplace=True)
female_data["Total_Unemployed"].fillna(female_data.groupby("Province")["Total_Unemployed"].transform("mean"), inplace=True)

In [53]:
male_data['Rounded_Total'] = male_data['Total_Unemployed'].round(decimals = 1)
female_data['Rounded_Total'] = female_data['Total_Unemployed'].round(decimals = 1)

In [64]:
male_values = male_data['Rounded_Total'].tolist()
female_values = female_data['Rounded_Total'].tolist()

In [78]:
print (len(male_values))
print (len(female_values))

13480
13480


In [100]:
rand_male = random.sample(male_values, 1000)
rand_female = random.sample(female_values, 1000)

In [101]:
print (len(rand_male))
print (len(rand_female))

1000
1000


In [102]:
#Computing the difference in mean
np.mean(rand_male) - np.mean(rand_female)

2.4231

In [103]:
p_value = permutation_test(rand_male, rand_female,
                           method='approximate',
                           num_rounds=10000,
                           seed=0)
print(p_value)

0.0004


Permutation tests (also called exact tests, randomization tests, or re-randomization tests) are nonparametric test procedures to test the null hypothesis that two different groups come from the same distribution. A permutation test can be used for significance or hypothesis testing (including A/B testing) without requiring to make any assumptions about the sampling distribution (e.g., it doesn't require the samples to be normal distributed).

Under the null hypothesis (treatment = control), any permutations are equally likely. (Note that there are (n+m)! permutations, where n is the number of records in the treatment sample, and m is the number of records in the control sample). For a two-sided test, we define the alternative hypothesis that the two samples are different (e.g., treatment != control).

Compute the difference (here: mean) of sample x and sample y <br>

Combine all measurements into a single dataset <br>

Draw a permuted dataset from all possible permutations of the dataset in 2. <br>

Divide the permuted dataset into two datasets x' and y' of size n and m, respectively <br>

Compute the difference (here: mean) of sample x' and sample y' and record this difference <br>

Repeat steps 3-5 until all permutations are evaluated <br>

Return the p-value as the number of times the recorded differences were more extreme than the original difference from 1. and divide this number by the total number of permutations
Here, the p-value is defined as the probability, given the null hypothesis (no difference between the samples) is true, that we obtain results that are at least as extreme as the results we observed (i.e., the sample difference from 1.).

#### Since p-value < alpha, we can reject the null hypothesis that the two samples come from the same distribution.